In [131]:
from PIL import Image
import os
import os.path
import pandas as pd
import torch.utils.data
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [132]:
def default_image_loader(path):
    return Image.open(path).convert('RGB')

In [133]:
class TripletImageLoader(torch.utils.data.Dataset):
    def __init__(self, base_path, filenames_filename, triplets_file_name, transform=None,
                 loader=default_image_loader):

        self.base_path = base_path  
        triplets = []
        data = np.array(pd.read_csv("triplets.csv"))
        self.triplets = data
        self.transform = transform
        self.loader = loader
    
    def __getitem__(self, index):
        img1 = self.loader(os.path.join(self.base_path,self.triplets[index][0]))
        img2 = self.loader(os.path.join(self.base_path,self.triplets[index][1]))
        img3 = self.loader(os.path.join(self.base_path,self.triplets[index][2]))
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3
            
    def __len__(self):
        return len(self.triplets) 

In [163]:
class bulid_data():
    def __init__(self):
        pass
    def read_data(self, mode, split):
        np.random.seed(100)
        if (mode == 'train'):
            data = np.array(pd.read_csv("train.csv"))
            self.all_classes = np.unique(data[:,1])
            file_list = np.arange(data.shape[0])
            np.random.shuffle(file_list)
            self.train = data[file_list[:int(data.shape[0]*split)]][:,0]
            self.train_labels = data[file_list[:int(data.shape[0]*split)]][:,1]
            self.valid = data[file_list[int(data.shape[0]*split):]][:,0]
            self.valid_labels = data[file_list[int(data.shape[0]*split):]][:,1]
                
        elif (mode == 'test'):
            data = pd.read_csv("test.csv")
            self.test = data['Image']
            
            
    def make_triplet_list(self, ntriplets, mode):
        print('Processing Triplet Generation ...')
        if (mode == "train"):
            np_labels = self.train_labels
            filename = "train_triplet_file.csv"
            
        elif (mode == "valid"):
            np_labels = self.valid_labels
            filename = "valid_triplet_file.csv"
        
        triplets = []
        for i in range(np_labels.shape[0]):
#             print(self.train[i])
            if (mode == 'train'):
                this_for_label = self.train_labels[i]
            else:
                this_for_label = self.valid_labels[i]
            anchor = i
            close = np.random.choice(np.where(np_labels == this_for_label)[0], 1, replace=True)
            count = 0
            while (close - i == 0 and count < 10):
                close = np.random.choice(np.where(np_labels == this_for_label)[0], 1, replace=True)
                count += 1
            far = np.random.choice(np.where(np_labels != this_for_label)[0], 1, replace=True)
            
            if (mode == 'train'):
                triplets.append([self.train[int(anchor)], self.train[int(close)], self.train[int(far)], self.train_labels[i] ]) 
            else:
                triplets.append([self.valid[int(anchor)], self.valid[int(close)], self.valid[int(far)], self.valid_labels[i] ]) 
#         return np.array(triplets)
        data = pd.DataFrame(np.array(triplets).reshape(-1,4),columns=['anchor', 'far', 'close', 'labels'])
        data.to_csv("triplets_"+str(mode)+".csv", index=False)

In [164]:
a = bulid_data()

In [165]:
a.read_data('train', 0.95)

In [167]:
a.make_triplet_list(300, 'valid')

Processing Triplet Generation ...
